# Checkpoint 1 - Análise Covid Brasil

## Carregar módulos

In [1]:
import numpy as np
import pandas as pd

## Carregar dados

- Dados Covid Brasil

In [2]:
! powershell Invoke-WebRequest https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-cities-time.csv.gz -o cases-brazil-cities-time.csv.gz

In [3]:
! 7z x cases-brazil-cities-time.csv.gz -aoa


7-Zip 23.01 (x64) : Copyright (c) 1999-2023 Igor Pavlov : 2023-06-20

Scanning the drive for archives:
1 file, 11465888 bytes (11 MiB)

Extracting archive: cases-brazil-cities-time.csv.gz
--
Path = cases-brazil-cities-time.csv.gz
Type = gzip
Headers Size = 44

Everything is Ok

Size:       33897495
Compressed: 11465888


In [4]:
dados_brasil = pd.read_csv('cases-brazil-cities-time_2023.csv')

In [5]:
dados_brasil

,epi_week,date,state,city,ibgeID,newDeaths,deaths,newCases,totalCases,deaths_per_100k_inhabitants,totalCases_per_100k_inhabitants,deaths_by_totalCases
0,301,2023-01-01,GO,Abadia de Goiás/GO,5200050,0,49,0,3952,535.051,43153.527,0.012
1,301,2023-01-01,MG,Abadia dos Dourados/MG,3100104,0,20,0,1929,284.819,27470.806,0.010
2,301,2023-01-01,GO,Abadiânia/GO,5200100,0,55,0,2154,263.498,10319.552,0.026
3,301,2023-01-01,PA,Abaetetuba/PA,1500107,0,243,0,11776,151.459,7339.861,0.021
4,301,2023-01-01,MG,Abaeté/MG,3100203,0,48,0,4130,206.336,17753.514,0.012
...,...,...,...,...,...,...,...,...,...,...,...,...
430964,311,2023-03-18,PR,Ângulo/PR,4101150,0,11,0,967,375.299,32992.153,0.011
430965,311,2023-03-18,BA,Érico Cardoso/BA,2900504,0,53,13,2467,504.138,23466.185,0.021
430966,311,2023-03-18,PA,Óbidos/PA,1505106,0,166,25,8717,316.353,16612.353,0.019
430967,311,2023-03-18,SP,Óleo/SP,3533809,0,6,0,162,245.198,6620.351,0.037


- Dados censo

In [6]:
dados_censo = pd.read_csv('dados_municipios_2010.csv', sep = ';', decimal = ',', encoding = 'latin1')

## Ajustar dados

In [7]:
max_date = max(dados_brasil['date'])

In [8]:
dados_brasil = dados_brasil[(dados_brasil['date'] == max_date) & \
                            (dados_brasil['state'] != 'TOTAL')]

In [9]:
cols = ['ibgeID', 'date', 'state', 'city', 'totalCases', 'deaths']

In [10]:
dados_brasil = dados_brasil[cols].reset_index(drop=True)

- Fórmula do índice de Gini

In [11]:
def gini(array):
    """Calculate the Gini coefficient of a numpy array."""
    # based on bottom eq:
    # http://www.statsdirect.com/help/generatedimages/equations/equation154.svg
    # from:
    # http://www.statsdirect.com/help/default.htm#nonparametric_methods/gini.htm
    # All values are treated equally, arrays must be 1d:
    array = array.flatten()
    if np.amin(array) < 0:
        # Values cannot be negative:
        array -= np.amin(array)
    # Values cannot be 0:
    array += 0.0000001
    # Values must be sorted:
    array = np.sort(array)
    # Index per array element:
    index = np.arange(1,array.shape[0]+1)
    # Number of array elements:
    n = array.shape[0]
    # Gini coefficient:
    return ((np.sum((2 * index - n  - 1) * array)) / (n * np.sum(array)))

## Aqui começa o trabalho

## GUSTAVO CESAR PINHEIRO DE AZEVEDO - RM: 550516

# Exercicio 1

In [12]:
cidades_dataset1 = dados_brasil['ibgeID'].nunique()
cidades_dataset1

5596

In [13]:
cidades_dataset2 = dados_censo['ibgeID'].nunique()
cidades_dataset2

5565

# Exercicio 2

Join

In [14]:
dados_brasil_censo = dados_brasil.merge(dados_censo, how='inner', on='ibgeID')
dados_brasil_censo = dados_brasil_censo[['ibgeID', 'date' ,'state', 'Municipio', 'totalCases', 'deaths', 'ESPVIDA', 'E_ANOSESTUDO', 'T_ANALF18M', 'RDPC', 'IDHM']]
dados_brasil_censo

,ibgeID,date,state,Municipio,totalCases,deaths,ESPVIDA,E_ANOSESTUDO,T_ANALF18M,RDPC,IDHM
0,5200050,2023-03-18,GO,ABADIA DE GOIÁS,4074,50,74.82,7.94,8.36,574.96,0.708
1,3100104,2023-03-18,MG,Abadia dos Dourados,1943,21,75.35,8.68,9.67,596.18,0.689
2,5200100,2023-03-18,GO,ABADIÂNIA,2218,55,75.46,9.31,11.67,519.87,0.689
3,1500107,2023-03-18,PA,ABAETETUBA,12009,244,72.89,8.99,14.97,293.01,0.628
4,3100203,2023-03-18,MG,Abaeté,4279,51,75.90,9.19,9.28,707.24,0.698
...,...,...,...,...,...,...,...,...,...,...,...
5560,4301552,2023-03-18,RS,ÁUREA,665,7,75.48,9.06,5.76,686.10,0.707
5561,4101150,2023-03-18,PR,ÂNGULO,967,11,74.62,9.54,12.11,641.37,0.721
5562,2900504,2023-03-18,BA,ÉRICO CARDOSO,2467,53,70.47,8.68,32.52,258.28,0.584
5563,1505106,2023-03-18,PA,ÓBIDOS,8717,166,70.35,8.89,13.73,255.03,0.594


ibgeID não localizados

In [15]:
ibgeID_n_localizados = dados_brasil.merge(dados_censo, how='outer',on='ibgeID', indicator=True)
ibgeID_n_localizados = ibgeID_n_localizados[ibgeID_n_localizados['_merge'] != 'both']
ibgeID_n_localizados = ibgeID_n_localizados[['ibgeID', 'state', 'city']]

ibgeID_n_localizados

,ibgeID,state,city
434,4220000,SC,Balneário Rincão/SC
781,12,AC,CASO SEM LOCALIZAÇÃO DEFINIDA/AC
782,27,AL,CASO SEM LOCALIZAÇÃO DEFINIDA/AL
783,13,AM,CASO SEM LOCALIZAÇÃO DEFINIDA/AM
784,16,AP,CASO SEM LOCALIZAÇÃO DEFINIDA/AP
785,29,BA,CASO SEM LOCALIZAÇÃO DEFINIDA/BA
786,23,CE,CASO SEM LOCALIZAÇÃO DEFINIDA/CE
787,32,ES,CASO SEM LOCALIZAÇÃO DEFINIDA/ES
788,52,GO,CASO SEM LOCALIZAÇÃO DEFINIDA/GO
789,21,MA,CASO SEM LOCALIZAÇÃO DEFINIDA/MA


# Exercicio 3

In [38]:
dados_brasil_censo = dados_brasil_censo.astype({'deaths': float, 'totalCases': float})
dados_brasil_censo

,ibgeID,date,state,Municipio,totalCases,deaths,ESPVIDA,E_ANOSESTUDO,T_ANALF18M,RDPC,IDHM
0,5200050,2023-03-18,GO,ABADIA DE GOIÁS,4074.0,50.0,74.82,7.94,8.36,574.96,0.708
1,3100104,2023-03-18,MG,Abadia dos Dourados,1943.0,21.0,75.35,8.68,9.67,596.18,0.689
2,5200100,2023-03-18,GO,ABADIÂNIA,2218.0,55.0,75.46,9.31,11.67,519.87,0.689
3,1500107,2023-03-18,PA,ABAETETUBA,12009.0,244.0,72.89,8.99,14.97,293.01,0.628
4,3100203,2023-03-18,MG,Abaeté,4279.0,51.0,75.90,9.19,9.28,707.24,0.698
...,...,...,...,...,...,...,...,...,...,...,...
5560,4301552,2023-03-18,RS,ÁUREA,665.0,7.0,75.48,9.06,5.76,686.10,0.707
5561,4101150,2023-03-18,PR,ÂNGULO,967.0,11.0,74.62,9.54,12.11,641.37,0.721
5562,2900504,2023-03-18,BA,ÉRICO CARDOSO,2467.0,53.0,70.47,8.68,32.52,258.28,0.584
5563,1505106,2023-03-18,PA,ÓBIDOS,8717.0,166.0,70.35,8.89,13.73,255.03,0.594


In [39]:
dados_brasil_censo.dtypes

ibgeID            int64
date             object
state            object
Municipio        object
totalCases      float64
deaths          float64
ESPVIDA         float64
E_ANOSESTUDO    float64
T_ANALF18M      float64
RDPC            float64
IDHM            float64
dtype: object

In [44]:
dados_gini = dados_brasil_censo.groupby('state').agg(
    casos_covid = pd.NamedAgg('totalCases', 'sum'),
    casos_gini = pd.NamedAgg('totalCases', aggfunc=lambda x: gini(np.array(x))),
    mortes_covid = pd.NamedAgg('deaths', 'sum'),
    mortes_gini = pd.NamedAgg('deaths', aggfunc=lambda x: gini(np.array(x)))
).reset_index()

Vemos uma notavel diferença no Distrito Federal, o que faz total sentido.
Dado a lógica do calculo, só teremos o resultado de 0.0 em conjuntos homogeneos, e como o UF do Distrito Federal (DF) possui apenas uma cidade, Brasilia, ficamos com um modelo homogeneo na analise

In [45]:
dados_gini

,state,casos_covid,casos_gini,mortes_covid,mortes_gini
0,AC,160678.0,0.686526,2044.0,0.718778
1,AL,337948.0,0.726772,7248.0,0.731252
2,AM,634562.0,0.670580,14457.0,0.826242
3,AP,185825.0,0.713223,2145.0,0.838724
4,BA,1772448.0,0.663848,31120.0,0.716423
5,CE,1438276.0,0.630618,28157.0,0.728672
6,DF,898072.0,0.000000,11845.0,0.000000
7,ES,1315879.0,0.638870,14762.0,0.677606
8,GO,1906503.0,0.769531,28057.0,0.796140
9,MA,493809.0,0.631114,11056.0,0.738336


# Exercicio 4

In [46]:
dados_brasil_censo.head(2)

,ibgeID,date,state,Municipio,totalCases,deaths,ESPVIDA,E_ANOSESTUDO,T_ANALF18M,RDPC,IDHM
0,5200050,2023-03-18,GO,ABADIA DE GOIÁS,4074.0,50.0,74.82,7.94,8.36,574.96,0.708
1,3100104,2023-03-18,MG,Abadia dos Dourados,1943.0,21.0,75.35,8.68,9.67,596.18,0.689


In [47]:
sumario_colunas = dados_brasil_censo.groupby('state').agg(
    CasosTotais = pd.NamedAgg('totalCases', 'sum'),
    MortesTotais = pd.NamedAgg('deaths', 'sum'),
    MediaESPVIDA = pd.NamedAgg('ESPVIDA', 'mean'),
    MediaE_ANOSESTUDO = pd.NamedAgg('E_ANOSESTUDO', 'mean'),
    MediaT_ANALF18M = pd.NamedAgg('T_ANALF18M', 'mean'),
    Mediana_IDHM = pd.NamedAgg('IDHM', 'median'),
    Mediana_RDPC = pd.NamedAgg('RDPC', 'median'),
    DP_RDPC = pd.NamedAgg('RDPC', 'std')
).reset_index()

sumario_colunas

,state,CasosTotais,MortesTotais,MediaESPVIDA,MediaE_ANOSESTUDO,MediaT_ANALF18M,Mediana_IDHM,Mediana_RDPC,DP_RDPC
0,AC,160678.0,2044.0,70.991818,7.752273,26.106818,0.5885,313.705,127.780499
1,AL,337948.0,7248.0,69.367843,8.896961,35.308529,0.5625,233.440,79.124345
2,AM,634562.0,14457.0,70.924677,7.451774,20.889194,0.5640,227.585,98.250134
3,AP,185825.0,2145.0,72.028125,9.035625,13.997500,0.6410,402.440,133.018963
4,BA,1772448.0,31120.0,70.570288,8.453453,25.816427,0.5890,272.860,101.927806
5,CE,1438276.0,28157.0,70.674076,9.675054,29.604728,0.6120,256.840,76.335807
6,DF,898072.0,11845.0,77.350000,9.870000,3.660000,0.8240,1715.110,NaN
7,ES,1315879.0,14762.0,74.536026,9.209231,13.271410,0.6835,533.375,203.473289
8,GO,1906503.0,28057.0,74.522683,9.800813,13.726098,0.6970,573.815,148.262560
9,MA,493809.0,11056.0,69.431198,9.057972,29.225945,0.5720,217.650,90.115351


# Exercicio 5

Identificamos que as cidades onde tiveram mais mortes são as que tendem a ter o desvio padrão mais alto (exceção DF).

Identificamos também que a expectativa de vida tem forte relação com o RDPC

In [60]:
ex5_df = sumario_colunas
ex5_df['Tx_Mortalidade'] = sumario_colunas['MortesTotais'] / sumario_colunas['CasosTotais']

ex5_df.sort_values(['MortesTotais'],ascending=False)

,state,CasosTotais,MortesTotais,MediaESPVIDA,MediaE_ANOSESTUDO,MediaT_ANALF18M,Mediana_IDHM,Mediana_RDPC,DP_RDPC,Tx_Mortalidade
25,SP,6506468.0,179297.0,75.153984,10.502357,8.100899,0.7380,686.890,197.398270,0.027557
18,RJ,2770310.0,76972.0,73.959348,8.955109,8.235543,0.7125,618.330,217.600292,0.027785
10,MG,4194616.0,65575.0,74.424771,9.092403,14.480703,0.6700,475.230,173.077762,0.015633
17,PR,2911560.0,45802.0,74.224536,10.503283,11.142356,0.7060,592.270,150.286289,0.015731
22,RS,2973437.0,41939.0,75.420948,10.239859,7.150887,0.7170,714.125,207.996885,0.014105
4,BA,1772448.0,31120.0,70.570288,8.453453,25.816427,0.5890,272.860,101.927806,0.017558
5,CE,1438276.0,28157.0,70.674076,9.675054,29.604728,0.6120,256.840,76.335807,0.019577
8,GO,1906503.0,28057.0,74.522683,9.800813,13.726098,0.6970,573.815,148.262560,0.014716
15,PE,1160995.0,22705.0,70.334216,8.967189,29.189459,0.5930,271.360,117.490310,0.019557
23,SC,1995513.0,22625.0,75.812116,10.296314,7.065631,0.7360,768.810,199.720910,0.011338
